In [1]:
import pickle
from sklearn import svm
from statistics import mean
from random import shuffle
import numpy as np
from sklearn.utils import shuffle as shf
from sklearn.neural_network import MLPClassifier

In [2]:
file_object = open('E:/Time Series/pra.pkl', 'rb')
data2 = pickle.load(file_object)
file_object.close()

In [3]:
for key, value in data2.items():
    print (key, len(value))

SOYABEENS 204639
CORN 151407
DEVELOPED 12214
FOREST 14646
WATER BODY 6501
ALFALFA 781
OATS 25


In [4]:
data = data2

In [5]:
data = {key:value[:6000] for key, value in data.items() if len(value) >= 6000}

In [6]:
for i in range(45):
    lin = [l[i] for value in data.values() for l in value]
    mins = min(lin)
    maxs = max(lin)
    means = mean(lin)
    for key, value in data.items():
        for example in value:
            example[i] = np.float64((example[i] - means)/(maxs-mins))

C:\Users\prabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in ushort_scalars
  


In [7]:
training_examples = []
training_labels = []
test_examples = []
test_labels = []
cv_examples = []
cv_labels = []
i = 0;
for key, value in data.items():
    training_examples = training_examples + value[:5000]
    cv_examples = cv_examples + value[5000:5500]
    test_examples = test_examples + value[5500:]
    training_labels = training_labels + [i for v in range(5000)]
    cv_labels = cv_labels + [i for v in range(500)]
    test_labels = test_labels + [i for v in range(500)]
    print(i,':', key)
    i+=1

0 : SOYABEENS
1 : CORN
2 : DEVELOPED
3 : FOREST
4 : WATER BODY


In [8]:
training_examples, training_labels  = shf(training_examples, training_labels)

In [10]:
clf = svm.SVC(C=1.0, 
              kernel='rbf', 
              degree= 3,
              gamma='auto',
              coef0=0.0, 
              shrinking=True, 
              probability=False, 
              tol=0.001, 
              cache_size=200, 
              class_weight=None, 
              verbose=False, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)


In [11]:
clf.fit(training_examples, training_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels)

0.5952

In [13]:
sum(clf.predict(training_examples) == training_labels)/len(training_labels)

0.73996

In [ ]:
sum(clf.predict(test_examples) == test_labels)/len(test_labels)

In [16]:
cnf = MLPClassifier(hidden_layer_sizes=(35,25,15),
                    activation='relu', 
                    solver='adam',
                    alpha=0.0001, 
                    batch_size='auto', 
                    learning_rate='constant',
                    learning_rate_init=0.001, 
                    power_t=0.5, 
                    max_iter=200, 
                    shuffle=True, 
                    random_state=None, 
                    tol=0.0001, 
                    verbose=True, 
                    warm_start=False, 
                    momentum=0.9, 
                    nesterovs_momentum=True, 
                    early_stopping=False, 
                    validation_fraction=0.1, 
                    beta_1=0.9, 
                    beta_2=0.999, 
                    epsilon=1e-08
                   )

In [17]:
cnf.fit(training_examples, training_labels)

Iteration 1, loss = 1.67210252
Iteration 2, loss = 1.32474324
Iteration 3, loss = 1.25548438
Iteration 4, loss = 1.22481218
Iteration 5, loss = 1.20692244
Iteration 6, loss = 1.19491155
Iteration 7, loss = 1.18350947
Iteration 8, loss = 1.17453686
Iteration 9, loss = 1.16700448
Iteration 10, loss = 1.15733070
Iteration 11, loss = 1.15175525
Iteration 12, loss = 1.14280619
Iteration 13, loss = 1.13340650
Iteration 14, loss = 1.12471092
Iteration 15, loss = 1.11822020
Iteration 16, loss = 1.11097372
Iteration 17, loss = 1.10330775
Iteration 18, loss = 1.09526727
Iteration 19, loss = 1.08659640
Iteration 20, loss = 1.07750329
Iteration 21, loss = 1.07016043
Iteration 22, loss = 1.05846518
Iteration 23, loss = 1.04712430
Iteration 24, loss = 1.03904937
Iteration 25, loss = 1.03193533
Iteration 26, loss = 1.02447429
Iteration 27, loss = 1.01729283
Iteration 28, loss = 1.01303218
Iteration 29, loss = 1.00791183
Iteration 30, loss = 1.00352053
Iteration 31, loss = 0.99898890
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(35, 25, 15), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [20]:
sum(cnf.predict(cv_examples) == cv_labels)/len(cv_labels)

0.6476

In [21]:
sum(cnf.predict(training_examples) == training_labels)/len(training_labels)

0.73708

In [ ]:
sum(cnf.predict(test_examples) == test_labels)/len(test_labels)